# Flow Theory

# IMPORTS

In [ ]:
from typing import List, Tuple
import math
from collections import Counter, deque
import urllib.request
import time

# UTILS

In [ ]:
def conv_seconds_milliseconds(seconds: float) -> float:
    return seconds * 1000

def duration(start: float, end: float) -> float:
    return end - start


# Max Flow

In [21]:
"""
FordFulkeron algorithm for maximum flow problem 
- pluggable augmenting path finding algorithms
- residual graph
- bottleneck capacity
- merge parallele edges by sum of all parallel edges capacities
- depth first search
- Edmonds-Karp algorithm
- capacity scaling algorithm
- dinics algorithm
"""
class FordFulkersonMaxFlowV1:
    """
    Ford-Fulkerson algorithm 
    - pluggable augmenting path finding algorithms
    - residual graph
    - bottleneck capacity
    """
    def __init__(self, n: int, edges: List[Tuple[int, int, int]]):
        self.size = n
        self.edges = edges

    def build(self, n: int, edges: List[Tuple[int, int, int]]) -> None:
        self.adj_list = {}
        self.delta = cnt = 0
        for u, v, cap in edges:
            if u not in self.adj_list:
                self.adj_list[u] = Counter()
            self.adj_list[u][v] += cap
            if v not in self.adj_list:
                self.adj_list[v] = Counter()
            self.delta = max(self.delta, self.adj_list[u][v])
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    def main_dfs(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.dfs(source, sink, math.inf)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def reset(self) -> None:
        self.parents = [-1] * self.size

    def dfs(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.parents[node] = 1
        cap = self.adj_list[node]
        for nei, cap in cap.items():
            if self.parents[nei] == -1 and cap > 0:
                cur_flow = self.dfs(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[node][nei] -= cur_flow
                    self.adj_list[nei][node] += cur_flow
                    return cur_flow
        return 0
    
    def main_edmonds_karp(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.edmonds_karp(source, sink)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def edmonds_karp(self, source: int, sink: int) -> int:
        queue = deque([(source, math.inf)])
        self.parents[source] = -2
        while queue:
            node, flow = queue.popleft()
            if node == sink:
                break
            capacity = self.adj_list[node]
            for nei, cap in capacity.items():
                if self.parents[nei] == -1 and cap > 0:
                    self.parents[nei] = node
                    queue.append((nei, min(flow, cap)))
        if node == sink:
            while node != source:
                parent = self.parents[node]
                self.adj_list[parent][node] -= flow
                self.adj_list[node][parent] += flow # residual edge
                node = parent
            return flow
        return 0

    def main_capacity_scaling(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.delta > 0:
            while True:
                self.reset()
                cur_flow = self.capacity_scaling(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
            self.delta >>= 1
        return maxflow

    def capacity_scaling(self, source: int, sink: int, flow: int) -> int:
        if source == sink:
            return flow
        self.parents[source] = 1
        capacity = self.adj_list[source]
        for nei, cap in capacity.items():
            if self.parents[nei] == -1 and cap >= self.delta:
                cur_flow = self.capacity_scaling(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[source][nei] -= cur_flow
                    self.adj_list[nei][source] += cur_flow
                    return cur_flow
        return 0

    def dinics_bfs(self, source: int, sink: int) -> bool:
        self.distances = [-1] * self.size
        self.distances[source] = 0
        queue = deque([source])
        while queue:
            node = queue.popleft()
            for nei, cap in self.adj_list[node].items():
                if self.distances[nei] == -1 and cap > 0:
                    self.distances[nei] = self.distances[node] + 1
                    queue.append(nei)
        return self.distances[sink] != -1

    def dinics_dfs(self, node: int, sink: int, flow: int) -> int:
        if flow == 0: return 0
        if node == sink: return flow
        for nei, cap in self.adj_list[node].items():
            if self.distances[nei] == self.distances[node] + 1 and cap > 0:
                cur_flow = self.dinics_dfs(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[node][nei] -= cur_flow
                    self.adj_list[nei][node] += cur_flow
                    return cur_flow
        return 0

    def main_dinics(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.dinics_bfs(source, sink):
            self.reset()
            while True:
                cur_flow = self.dinics_dfs(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
        return maxflow

In [22]:
"""
FordFulkeron algorithm for maximum flow problem 
- pluggable augmenting path finding algorithms
- residual graph
- bottleneck capacity
- flow edge class
- depth first search
- Edmonds-Karp algorithm
- capacity scaling algorithm
- dinics algorithm
-- deadend elimination
"""
class FlowEdge:
    def __init__(self, src: int, dst: int, cap: int):
        self.src = src # source node
        self.dst = dst # destination node
        self.cap = cap

    def __repr__(self):
        return f'source node: {self.src}, destination node: {self.dst}, capacity: {self.cap} ======'

class FordFulkersonMaxFlowV2:
    """
    Ford-Fulkerson algorithm 
    - pluggable augmenting path finding algorithms
    - residual graph
    - bottleneck capacity
    """
    def __init__(self, n: int, edges: List[Tuple[int, int, int]]):
        self.size = n
        self.edges = edges

    def build(self, n: int, edges: List[Tuple[int, int, int]]) -> None:
        self.flowedges = []
        self.adj_list = {}
        self.delta = 0
        for u, v, cap in edges:
            self.flowedges.append(FlowEdge(u, v, cap))
            if u not in self.adj_list:
                self.adj_list[u] = []
            self.adj_list[u].append(len(self.flowedges) - 1)
            self.flowedges.append(FlowEdge(v, u, 0)) # residual edge
            if v not in self.adj_list:
                self.adj_list[v] = []
            self.adj_list[v].append(len(self.flowedges) - 1)
            self.delta = max(self.delta, cap)
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    def main_dfs(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.dfs(source, sink, math.inf)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def reset(self) -> None:
        self.vis = [0] * self.size

    def neighborhood(self, node: int) -> List[int]:
        return (i for i in self.adj_list[node])

    def dfs(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.vis[node] = 1
        for index in self.neighborhood(node):
            nei = self.flowedges[index]
            if self.vis[nei.dst] == 0 and nei.cap > 0:
                cur_flow = self.dfs(nei.dst, sink, min(flow, nei.cap))
                if cur_flow > 0:
                    nei.cap -= cur_flow
                    self.flowedges[index ^ 1].cap += cur_flow
                    return cur_flow
        return 0
    
    def main_edmonds_karp(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            self.parents = [-1] * len(self.flowedges)
            cur_flow = self.edmonds_karp(source, sink)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def edmonds_karp(self, source: int, sink: int) -> int:
        queue = deque([(source, math.inf, -1)])
        self.vis[source] = 1
        while queue:
            node, flow, prev_index = queue.popleft()
            if node == sink:
                break
            for index in self.neighborhood(node):
                nei = self.flowedges[index]
                if self.vis[nei.dst] == 0 and nei.cap > 0:
                    self.vis[nei.dst] = 1
                    self.parents[index] = prev_index
                    queue.append((nei.dst, min(flow, nei.cap), index))
        if node == sink:
            while prev_index != -1:
                parent_index = self.parents[prev_index]
                self.flowedges[prev_index].cap -= flow
                self.flowedges[prev_index^1].cap += flow # residual edge
                prev_index = parent_index
            return flow
        return 0

    def main_capacity_scaling(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.delta > 0:
            while True:
                self.reset()
                cur_flow = self.capacity_scaling(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
            self.delta >>= 1
        return maxflow

    def capacity_scaling(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.vis[node] = 1
        for index in self.neighborhood(node):
            nei = self.flowedges[index]
            if self.vis[nei.dst] == 0 and nei.cap >= self.delta:
                cur_flow = self.capacity_scaling(nei.dst, sink, min(flow, nei.cap))
                if cur_flow > 0:
                    nei.cap -= cur_flow
                    self.flowedges[index ^ 1].cap += cur_flow
                    return cur_flow
        return 0

    def dinics_bfs(self, source: int, sink: int) -> bool:
        self.distances = [-1] * self.size
        self.distances[source] = 0
        queue = deque([source])
        while queue:
            node = queue.popleft()
            for index in self.neighborhood(node):
                nei = self.flowedges[index]
                if self.distances[nei.dst] == -1 and nei.cap > 0:
                    self.distances[nei.dst] = self.distances[node] + 1
                    queue.append(nei.dst)
        return self.distances[sink] != -1

    def dinics_dfs(self, node: int, sink: int, flow: int) -> int:
        if flow == 0: return 0
        if node == sink: return flow
        while self.ptr[node] < len(self.adj_list[node]):
            index = self.adj_list[node][self.ptr[node]]
            self.ptr[node] += 1
            nei = self.flowedges[index]
            if self.distances[nei.dst] == self.distances[node] + 1 and nei.cap > 0:
                cur_flow = self.dinics_dfs(nei.dst, sink, min(flow, nei.cap))
                if cur_flow > 0:
                    nei.cap -= cur_flow
                    self.flowedges[index ^ 1].cap += cur_flow
                    return cur_flow
        return 0

    def main_dinics(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.dinics_bfs(source, sink):
            self.reset()
            self.ptr = [0] * self.size # pointer to the next edge to be processed (optimizes for dead ends)
            while True:
                cur_flow = self.dinics_dfs(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
        return maxflow

# [Download Speed](https://cses.fi/problemset/task/1694/)

In [23]:
urls = ['https://cses.fi/file/acec992e42fe3462f07114ad2d5f7ce9ff27434922e9b52f39006310ca79d019/1/1/', \
    'https://cses.fi/file/558f035a5dce8931e19371bda522b5a81d28a9a1a1835a6205e566ca9de324c8/1/1/', \
    'https://cses.fi/file/9201642e4901d251a2c18f26429a67089a018a07cd3aa6025cb5fd12d4f88126/1/1/', \
    'https://cses.fi/file/654fbbbac2b61ff15187c1d399394ea7e27b05b3dddf32bdba1bb1c6708e3593/1/1/', \
    'https://cses.fi/file/8286fe339a5312417d20620138dec793deb78cd8960f33ddc4f521982e71f046/1/1/', \
    'https://cses.fi/file/297a2fce46a4102cbd86bea796751acd566fcae258aa00b62d34f5436e441b27/1/1/', \
    'https://cses.fi/file/f1cb0fbf03699e8e91a47846d49e084dae8ec899186d7766461383b5bf562452/1/1/', \
    'https://cses.fi/file/d31400a9196af8d78037127201e471353fcd1f5aaecc9939ea4740a054559c0f/1/1/', \
    'https://cses.fi/file/963201f693af2a27f8d43a78a6213b938576971e71fd5270ad62b538cae9cd47/1/1/', \
    'https://cses.fi/file/9b1a8c894a16cc3228c663a38b764156f7f47183b2f7b206866f935d693dbae7/1/1/', \
    'https://cses.fi/file/e27523c04940efd4cddc19cb7ad99a65635c2fb88cf1c86a7492e08089e8c942/1/1/', \
    'https://cses.fi/file/a09e3665a05e05a0f4e6d590b271ba889bed02c5aae69522d38d8bf1c62aa371/1/1/', \
    'https://cses.fi/file/ec19840ed099c8e55fd77bf40b1cf4f6fdbd43c0a63c74dfe736de4d38cb67cd/1/1/']

In [ ]:
"""
Using the dfs implementation as the base case, it was tested to work in the online judge.
"""
results = [0]*len(urls)
for i, url in enumerate(urls):
    data = urllib.request.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    start_time = time.perf_counter()
    mf = FordFulkersonMaxFlowV2(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    results[i] = mf
    print(f'Finished testcase: {i} in {end_time - start_time} seconds')

In [24]:
for i, url in enumerate(urls):
    data = urllib.request.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    start_time = time.perf_counter()
    mf_dfs1 = FordFulkersonMaxFlowV1(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    duration_dfs1 = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_edmonds1 = FordFulkersonMaxFlowV1(n, edges).main_edmonds_karp(0, n - 1)
    end_time = time.perf_counter()
    duration_edmonds_karp1 = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_cp_scaling1 = FordFulkersonMaxFlowV1(n, edges).main_capacity_scaling(0, n - 1)
    end_time = time.perf_counter()
    duration_cp_scaling1 = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_dinics1 = FordFulkersonMaxFlowV1(n, edges).main_dinics(0, n - 1)
    end_time = time.perf_counter()
    duration_dinics1 = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_dfs = FordFulkersonMaxFlowV2(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    duration_dfs = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_edmonds = FordFulkersonMaxFlowV2(n, edges).main_edmonds_karp(0, n - 1)
    end_time = time.perf_counter()
    duration_edmonds_karp = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_cp_scaling = FordFulkersonMaxFlowV2(n, edges).main_capacity_scaling(0, n - 1)
    end_time = time.perf_counter()
    duration_cp_scaling = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_dinics = FordFulkersonMaxFlowV2(n, edges).main_dinics(0, n - 1)
    end_time = time.perf_counter()
    duration_dinics = duration(start_time, end_time)
    assert mf_edmonds1 == results[i], f'Failed on testcase: {i}, output: {mf_edmonds1}, expected: {results[i]} for edmonds-karp algorithm v1'
    assert mf_cp_scaling1 == results[i], f'Failed on testcase: {i}, output: {mf_cp_scaling1}, expected: {results[i]} for capacity scaling algorithm v1'
    assert mf_dinics1 == results[i], f'Failed on testcase: {i}, output: {mf_dinics1}, expected: {results[i]} for dinics algorithm v1'
    assert mf_edmonds == results[i], f'Failed on testcase: {i}, output: {mf_edmonds}, expected: {results[i]} for edmonds-karp algorithm v2'
    assert mf_cp_scaling == results[i], f'Failed on testcase: {i}, output: {mf_cp_scaling}, expected: {results[i]} for capacity scaling algorithm v2'
    assert mf_dinics == results[i], f'Failed on testcase: {i}, output: {mf_dinics}, expected: {results[i]} for dinics algorithm v2'
    print(f'dfs v1: {conv_seconds_milliseconds(duration_dfs1)} milliseconds')
    print(f'edmonds-karp v1: {conv_seconds_milliseconds(duration_edmonds_karp1)} milliseconds')
    print(f'capacity scaling v1: {conv_seconds_milliseconds(duration_cp_scaling1)} milliseconds')
    print(f'dinics v1: {conv_seconds_milliseconds(duration_dinics1)} milliseconds')
    print(f'dfs v2: {conv_seconds_milliseconds(duration_dfs)} milliseconds')
    print(f'edmonds-karp v2: {conv_seconds_milliseconds(duration_edmonds_karp)} milliseconds')
    print(f'capacity scaling v2: {conv_seconds_milliseconds(duration_cp_scaling)} milliseconds')
    print(f'dinics v2: {conv_seconds_milliseconds(duration_dinics)} milliseconds')
    print(f'============================================================== Test Case {i} Passed ==============================================================')


dfs v1: 0.07919800009403843 milliseconds
edmonds-karp v1: 0.05266600010145339 milliseconds
capacity scaling v1: 0.06254300001273805 milliseconds
dinics v1: 0.05668299991157255 milliseconds
dfs v2: 0.07072100015648175 milliseconds
edmonds-karp v2: 0.05720900003325369 milliseconds
capacity scaling v2: 0.06564099999195605 milliseconds
dinics v2: 0.06733700001859688 milliseconds
============================================================== Test Case 0 Passed ==============================================================
dfs v1: 0.04239800000505056 milliseconds
edmonds-karp v1: 0.02353600007154455 milliseconds
capacity scaling v1: 0.02175999998144107 milliseconds
dinics v1: 0.02424000012979377 milliseconds
dfs v2: 0.04455100020095415 milliseconds
edmonds-karp v2: 0.02800100014610507 milliseconds
capacity scaling v2: 0.02737899990279402 milliseconds
dinics v2: 0.0314809999508725 milliseconds
============================================================== Test Case 1 Passed ==================